# Data Science - Laboratorio 9
## Visualizaciones interactivas y dashboards
### Modelos LSTM
---
**Integrantes**
- Diego Alberto Leiva
- José Pablo Orellana

## Librerias

In [6]:
# Manipulacion de Datos
import pandas as pd
import numpy as np

# Sistema
import os

# Utilidades
import random

# Preprocesamiento
from sklearn.preprocessing import StandardScaler

# PyTorch CUDA
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

### Carga de Datos Limpios

In [7]:
# Leer los archivos de una carpeta y almacenarlos como dataframes
consumos = pd.read_csv('data/consumos_clean.csv', sep=';', encoding='utf-8')
importaciones = pd.read_csv('data/importaciones_clean.csv', sep=';', encoding='utf-8')
precios = pd.read_csv('data/precios_clean.csv', sep=';', encoding='utf-8')

## Modelos LSTM

### Configuracion de PyTorch con CUDA

Se verifica si el sistema tiene capacidad de CUDA o no

In [8]:
# Configurar la semilla para la reproducibilidad
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

if torch.cuda.is_available():
    print(f"CUDA AVAILABLE")

    print(f"Using: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    # Configurar el generador de números aleatorios de CUDA
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU.

    # Configurar el dispositivo de PyTorch para usar la GPU
    device = torch.device("cuda")

    # Configurar PyTorch a determinista
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
else:
    # Si no hay GPU disponible, usar la CPU
    print("CUDA NOT AVAILABLE")
    print("Using CPU")
    device = torch.device("cpu")

CUDA AVAILABLE
Using: NVIDIA GeForce RTX 3070 Ti Laptop GPU


### Arquitectura de una LSTM con PyTorch CUDA

In [9]:
class LSTM_Model():
    """
    Clase para entrenar un modelo LSTM para predecir series temporales.

    Args:
        input_size (int): Número de features en la entrada.
        output_size (int): Número de features en la salida.
        hidden_layers (int): Número de capas ocultas.
        num_layers (int): Número de capas LSTM.
        lookback (int): Número de pasos de tiempo hacia atrás.
        n_epochs (int): Número de épocas.
        test_size (int): Tamaño del conjunto de prueba.
        device (torch.device): Dispositivo de PyTorch.
        dataframe (pandas.DataFrame): Dataframe de entrada.
        target_column (str): Columna objetivo.

    Attributes:
        input_size (int): Número de features en la entrada.
        output_size (int): Número de features en la salida.
        hidden_layers (int): Número de capas ocultas.
        num_layers (int): Número de capas LSTM.
        lookback (int): Número de pasos de tiempo hacia atrás.
        n_epochs (int): Número de épocas.
        test_size (int): Tamaño del conjunto de prueba.
        device (torch.device): Dispositivo de PyTorch.
        dataframe (pandas.DataFrame): Dataframe de entrada.
        target_column (str): Columna objetivo.
        model (torch.nn.Module): Modelo LSTM.
        loss_fn (torch.nn.Module): Función de pérdida.
        optimizer (torch.optim.Optimizer): Optimizador.
        loss_train (list): Lista para almacenar la pérdida de entrenamiento.
        loss_test (list): Lista para almacenar la pérdida de prueba.
        train_size (int): Tamaño del conjunto de entrenamiento.
    """
    def __init__(self, input_size, output_size, hidden_layers, num_layers, lookback, n_epochs, 
                 test_size, device, dataframe, target_column, train_log, train_res):
        
        # Inicializar los hiperparámetros
        self.input_size = input_size            # Número de features en la entrada
        self.output_size = output_size          # Número de features en la salida
        self.hidden_layers = hidden_layers      # Número de capas ocultas
        self.num_layers = num_layers            # Número de capas LSTM
        self.lookback = lookback                # Número de pasos de tiempo hacia atrás
        self.n_epochs = n_epochs                # Número de épocas
        self.test_size = test_size              # Tamaño del conjunto de prueba
        self.device = device                    # Dispositivo de PyTorch
        self.dataframe = dataframe              # Dataframe de entrada
        self.target_column = target_column      # Columna objetivo
        self.train_log = train_log              # Booleano para guardar el log del entrenamiento
        self.train_res = train_res              # Booleano para guardar los resultados del entrenamiento

        # Crear el modelo LSTM 
        self.model = self.build_lstm_model()

        # Definir la función de pérdida y el optimizador
        self.loss_fn = nn.MSELoss().to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        # Listas para almacenar la pérdida
        self.loss_train = []
        self.loss_test = []

        # Entrenar el modelo
        self.train_model(dataframe, target_column)


    def build_lstm_model(self):
        """
        Función para construir el modelo LSTM.

        Returns:
            model (torch.nn.Module): El modelo LSTM.
        """
        class LSTM(nn.Module):
            def __init__(self, input_size, hidden_layers, output_size, num_layers):
                super(LSTM, self).__init__()
                
                # Definir la capa LSTM
                self.lstm = nn.LSTM(input_size = input_size, 
                                    hidden_size = hidden_layers, 
                                    num_layers = num_layers, 
                                    batch_first = True)
                
                # Definir la capa de salida
                self.linear = nn.Linear(hidden_layers, 
                                        output_size)
                
                # Agregar una capa de activación ReLU
                self.relu = nn.ReLU() 
                
                self.hidden_size = hidden_layers  # Numero de hidden states
                self.num_layers = num_layers    # Numero de capas
                

            def forward(self, x):
                # Inicializar los hidden states
                h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
                c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)

                # Forward pass a la capa LSTM
                out, _ = self.lstm(x, (h0, c0))

                # Aplicar la capa de activación
                out = self.relu(out)
                
                # Solo se necesita el último hidden state
                out = self.linear(out)
                
                return out
            
        # Retornar el modelo LSTM con los hiperparámetros dados
        return LSTM(self.input_size, self.hidden_layers, self.output_size, self.num_layers).to(self.device)
    


    def create_dataset(self, dataset):
        """
        Función para crear el dataset de entrenamiento.

        Args:
            dataset (numpy.array): El dataset de entrada.

        Returns:
            X (torch.Tensor): El tensor de features.
            y (torch.Tensor): El tensor de target.
        """
        X, y = [], []
        for i in range(len(dataset) - self.lookback):
            feature = dataset[i : i + self.lookback]
            target = dataset[i + 1 : i + self.lookback + 1]
            X.append(feature)
            y.append(target)

        # Convertir listas a numpy arrays y luego a tensores
        X = np.array(X)
        y = np.array(y)
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
    

    def train_model(self, dataframe, variable, ):
        """
        Función para entrenar el modelo LSTM.

        Args:
            dataframe (pandas.DataFrame): El dataframe de entrada.

        Returns:
            None
        """
        # Copiar el dataframe
        df = dataframe.copy()
        # Seleccionar la variable objetivo
        timeseries = df[[variable]].values.astype(float)
        # Normalizar los datos
        scaler = StandardScaler()
        timeseries = scaler.fit_transform(timeseries)

        # Dividir los datos en conjuntos de entrenamiento y prueba
        self.train_size = len(timeseries) - self.test_size
        train_data = timeseries[:self.train_size]
        test_data = timeseries[self.train_size:]

        # Crear el dataset de entrenamiento y prueba 
        X_train, y_train = self.create_dataset(train_data)
        X_test, y_test = self.create_dataset(test_data)

        # Mover los tensores al dispositivo
        X_train, y_train = X_train.to(self.device), y_train.to(self.device)
        X_test, y_test = X_test.to(self.device), y_test.to(self.device)

        # Crear un DataLoader para cargar los datos
        loader = data.DataLoader(data.TensorDataset(X_train, y_train), batch_size=16, shuffle=False)

        # Iteramos sobre cada epoca
        for epoch in range(self.n_epochs):
            # Colocamos el modelo en modo de entrenamiento
            self.model.train()

            # Cargamos los batches
            for X_batch, y_batch in loader:
                # Movemos los datos al dispositivo
                X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)

                # Obtenemos una primera prediccion
                y_pred = self.model(X_batch)
                # Calculamos la perdida
                loss = self.loss_fn(y_pred, y_batch)
                # Reseteamos la gradiente a cero
                #   sino la gradiente de previas iteraciones se acumulará con las nuevas
                self.optimizer.zero_grad()
                # Backpropagation
                loss.backward()
                # Aplicar las gradientes para actualizar los parametros del modelo
                self.optimizer.step()

            # Validación cada 100 epocas
            if epoch % 25 == 0 or epoch == self.n_epochs - 1:
                # Colocamos el modelo en modo de evaluación
                self.model.eval()

                # Deshabilitamos el calculo de gradientes
                with torch.no_grad():
                    # Prediccion de train
                    y_pred = self.model(X_train)
                    # Calculo del RMSE - Root Mean Square Error
                    train_rmse = np.sqrt(self.loss_fn(y_pred, y_train).cpu().numpy())
                    # Prediccion sobre validation
                    y_pred = self.model(X_test)
                    # Calculo del RMSE para validation
                    test_rmse = np.sqrt(self.loss_fn(y_pred, y_test).cpu().numpy())
                    # Almacenamos los resultados
                    self.loss_train.append(train_rmse)
                    self.loss_test.append(test_rmse)
                
                if self.train_log:
                    # Mostramos los resultados
                    print(f"Epoch {epoch}: train RMSE {train_rmse:.4f}, test RMSE {test_rmse:.4f}")

            # Si hay GPU, sincronizamos el dispositivo
            if torch.cuda.is_available():
                torch.cuda.synchronize()

        # Guardar el las predicciones en un DataFrame
        result_df = self.save_results(timeseries, X_train, X_test)

        if self.train_res:
            # Guardar el DataFrame en un archivo CSV si es necesario
            result_df.to_csv(f'models/predictions_{self.target_column}.csv', index=False)
            print(f"Predicciones guardadas en predictions_{self.target_column}.csv")
        else:
            print("Predicciones no guardadas en CSV, solo retornadas como DataFrame.")

        return result_df

    def save_results(self, timeseries, X_train, X_test):
        """
        Función para guardar los resultados en un DataFrame.

        Args:
            timeseries (numpy.array): La serie temporal original.
            X_train (torch.Tensor): El tensor de features de entrenamiento.
            X_test (torch.Tensor): El tensor de features de prueba.

        Returns:
            result_df (pandas.DataFrame): DataFrame con la serie original y las predicciones.
        """

        with torch.no_grad():
            # Movemos las predicciones de entrenamiento
            train_plot = np.ones_like(timeseries) * np.nan
            # Predicciones de entrenamiento
            y_train_pred = self.model(X_train)
            # Tomamos solo el último paso de tiempo
            train_plot[self.lookback:self.train_size] = y_train_pred[:, -1, :].cpu().numpy().flatten()

            # Movemos las predicciones de prueba
            test_plot = np.ones_like(timeseries) * np.nan
            # Predicciones de prueba
            y_test_pred = self.model(X_test)
            test_plot[self.train_size + self.lookback:len(timeseries)] = y_test_pred[:, -1, :].cpu().numpy().flatten()

            # Crear el DataFrame con la serie temporal original y las predicciones
            result_df = pd.DataFrame({
                'Original Series': timeseries.flatten(),
                'Train Predictions': train_plot.flatten(),
                'Test Predictions': test_plot.flatten()
            })

        # Retornamos el DataFrame con los resultados
        return result_df

### Generando Modelo